In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper
import copy

In [ ]:
class Turtle(object):
    
    def __init__(self, loc=Point(0,0), rad=0, pen='up', delta_rad=np.pi/6, parent=None):
        self.loc = loc
        self.rad = rad
        self.pen = pen
        self.tree_depth = 0
        self.delta_rad = np.pi/6
        self.parent = parent
        self.alive = True
        self.lines = MultiLineString()
        self.pts_in_current_line = []
        
    def __repr__(self):
         return f'''
         Cowabunga it is
         pos=({self.x}, {self.y})
         pen={self.pen}
         rad={self.rad}
         tree_depth={self.tree_depth}'''
        
    @property
    def x(self):
        return self.loc.x
    
    @x.setter
    def x(self, x):
        self.loc = Point(x, self.y)
    
    @property
    def y(self):
        return self.loc.y
    
    @y.setter
    def y(self, y):
        self.loc = Point(self.x, y)
    
    @property
    def degrees(self):
        return self.rad / (np.pi * 2) * 360
    
    @degrees.setter
    def degrees(self, degrees):
        self.rad = degrees / 360 * np.pi * 2
    
    @property
    def current_line(self):
        if len(self.pts_in_current_line) > 1:
            return LineString(self.pts_in_current_line)
        
    @current_line.setter
    def current_line(self, current_line):
        self.pts_in_current_line = [Point(p) for p in current_line]
    
    def add_point(self):
        self.pts_in_current_line.append(self.loc)
        
    def pen_down(self):
        self.pen = 'down'
        self.add_point()
        
    def pen_up(self):
        self.pen = 'up'
        self.finish_line()
        
    def finish_line(self):
        self.lines = gp.merge_LineStrings([self.lines, self.current_line])
        self.pts_in_current_line = []
        
    def forward(self, d):
        self.x += np.cos(self.rad) * d
        self.y += np.sin(self.rad) * d
        if self.pen == 'down':
            self.add_point()
            
    def F(self, d=1):
        self.pen_down()
        self.forward(d)
        
    def f(self, d=1):
        self.pen_up()
        self.forward(d)
            
    def turn(self, d_angle, use_degrees=False):
        if use_degrees:
            new_degrees = self.degrees + d_angle
            self.degrees = new_degrees % 360
        else:
            new_rad = self.rad + d_angle
            self.rad = new_rad % (np.pi * 2)
        
    def plus(self):
        self.turn(self.delta_rad)
        
    def minus(self):
        self.turn(-self.delta_rad)
    
    def circle(self, radius, extent=(np.pi*2), n_eval_points=1000, use_degrees=False):
        
        d_angle = extent / n_eval_points
        forward_d = abs(d_angle * radius)
        if use_degrees:
            forward_d = forward_d / 360 * (2 * np.pi)
        for n in range(n_eval_points):
            self.forward(forward_d)
            self.turn(d_angle, use_degrees=use_degrees)

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
start_pt = gp.get_random_point_in_polygon(drawbox)
t = Turtle(start_pt)
t.pen_down()
t.circle(
        np.random.uniform(0.3,20), 
        np.random.uniform(-120, 120), 
        use_degrees=True,
        n_eval_points=20,
    )
future_turtles = [copy.deepcopy(t) for i in range(n_checks_per_iter)]

for ft in future_turtles:
    ft.circle(
        np.random.uniform(0.3,20), 
        np.random.uniform(-120, 120), 
        use_degrees=True,
        n_eval_points=20,
    )

In [ ]:
existing_line = t.current_line
ft = copy.deepcopy(t)

In [ ]:
n_eval_points = 40

In [ ]:
ft.circle(
        5, 
        290, 
        use_degrees=True,
        n_eval_points=n_eval_points,
    )
# ft.forward(10)

In [ ]:
existing_line.buffer(0.05, cap_style=2, join_style=2)

In [ ]:
ft.current_line[-20:].buffer(0.05, cap_style=2, join_style=2)

In [ ]:
t.current_line[:-8]

In [ ]:
n_checks_per_iter = 15
n_iters = 200
all_lines = MultiLineString()
all_lines_buffered = Polygon()
n_turtles = 3
n_eval_points = 16
for kk in range(n_turtles):
    start_pt = gp.get_random_point_in_polygon(drawbox)
    t = Turtle(start_pt)
    t.pen_down()
    t.circle(
        np.random.uniform(0.3,20), 
        np.random.uniform(-120, 120), 
        use_degrees=True,
        n_eval_points=20,
    )
    
    for jj in tqdm(range(n_iters)):
        existing_line = t.current_line.buffer(0.05, cap_style=2, join_style=2, resolution=1)
        future_turtles = [copy.deepcopy(t) for i in range(n_checks_per_iter)]
        new_lines = []
        for ft in future_turtles:
            ft.circle(
                np.random.uniform(1, 10.1), 
                np.random.uniform(-120, 120), 
                use_degrees=True,
                n_eval_points=n_eval_points,
            )
            new_line = LineString(ft.pts_in_current_line[-(n_eval_points)+1:]).buffer(0.05, cap_style=2, join_style=2, resolution=1)
            new_lines.append(new_line)
        
        distances = np.array([ft.loc.distance(existing_line) for ft in future_turtles])
        distance_ranks = np.argsort(distances)
        no_cross_current = ~np.array([new_line.intersects(existing_line) for new_line in new_lines])
        not_too_close = distances > 1
        in_drawbox = [ft.loc.within(drawbox) for ft in future_turtles]
        no_cross_others = ~np.array([new_line.intersects(all_lines_buffered) for new_line in new_lines])
        okay = no_cross_current & in_drawbox & not_too_close & no_cross_others
        if okay.sum() < 1:
            break
        winner = distance_ranks[okay][0]
        t = future_turtles[winner]
        
    t.pen_up()
        
    all_lines = gp.merge_LineStrings([all_lines, t.lines])
    all_lines_buffered = all_lines.buffer(0.05, cap_style=2, join_style=2, resolution=1)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(all_lines)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')
# sk.vpype('linesimplify --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.display()

# iteration not repetition

In [ ]:
n_iters = 150
std = 0.0008

# c = Point((0,0)).buffer(20, resolution=120).boundary
sq = Point((0,0)).buffer(20, resolution=1).boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 200)]).boundary
rings = [c]
for i in range(n_iters):
    current_ring = np.array([x for x in c.coords])
    current_ring *= np.random.normal(loc=1, scale=std, size=current_ring.shape)
    current_ring += np.ones_like(current_ring) * np.array([0.5, -0.1])
    c = LineString(current_ring)
    c = sa.rotate(c, -0.5)
    c = sa.scale(c, xfact=1.006, yfact=1.005)
    rings.append(c)

In [ ]:
mls = MultiLineString([r for r in rings])
layer = gp.make_like(mls, drawbox)

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(layer)
sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/191_do_not_repeat.svg')

## try 2

In [ ]:
# make page
paper_size = '17x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
n_iters = 250
low = 0.9999
high = 1.0001

# c = Point((0,0)).buffer(20, resolution=120).boundary
sq = Point((0,0)).buffer(20, resolution=1).boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 200)]).boundary
rings = [c]
rands = np.random.uniform(low, high, size=current_ring.shape)
for i in range(n_iters):
    current_ring = np.array([x for x in c.coords])
    current_ring *= rands
    current_ring += np.ones_like(current_ring) * np.array([0.5, -0.2])
    c = LineString(current_ring)
    c = sa.rotate(c, 0.6)
    c = sa.scale(c, xfact=1.006, yfact=1.008)
    rings.append(c)

In [ ]:
mls = MultiLineString([r for r in rings])
layer = gp.make_like(mls, drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(layer)
sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/192_do_not_repeat.svg')

## try 3

In [ ]:
# make page
paper_size = '17x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
sq = Point((0,0)).buffer(20, resolution=1).boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 200)]).boundary

In [ ]:
mu=0
sigma=0.01
rands = np.random.lognormal(mean=mu, sigma=sigma, size=current_ring.shape)
plt.plot(rands.ravel())

In [ ]:
n_iters = 280
low = 0.9999
high = 1.0002
mu=0
sigma=0.00015

# c = Point((0,0)).buffer(20, resolution=120).boundary
sq = Point((0,0)).buffer(20, resolution=5).boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 80)]).boundary
rings = [c]
current_ring = np.array([x for x in c.coords])
rands = np.random.uniform(low, high, size=current_ring.shape)
# rands = np.random.lognormal(mean=mu, sigma=sigma, size=current_ring.shape)
for i in range(n_iters):
    current_ring = np.array([x for x in c.coords])
    current_ring *= rands
    current_ring += np.ones_like(current_ring) * np.array([1.7, -0.6])
    c = LineString(current_ring)
#     c = sa.skew(c, xs=0.1)
    c = sa.rotate(c, 0.7)
    c = sa.scale(c, xfact=1.004, yfact=1.014)
    
    rings.append(c)

In [ ]:
mls = MultiLineString([r for r in rings])
layer = gp.make_like(mls, drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(layer)
# sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/194_do_not_repeat.svg')

In [ ]:
n_iters = 480
low = 0.9999
high = 1.0002
mu=0
sigma=0.00015

# c = Point((0,0)).buffer(20, resolution=120).boundary
sq = Point((0,0)).buffer(20, resolution=1).boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 80)]).boundary
rings = [c]
current_ring = np.array([x for x in c.coords])
rands = np.random.uniform(low, high, size=current_ring.shape)
# rands = np.random.lognormal(mean=mu, sigma=sigma, size=current_ring.shape)
for i in range(n_iters):
    current_ring = np.array([x for x in c.coords])
    current_ring *= rands
    current_ring += np.ones_like(current_ring) * np.array([0.3, -0.03])
    c = LineString(current_ring)
#     c = sa.skew(c, xs=0.1)
    c = sa.rotate(c, 0.6)
    c = sa.scale(c, xfact=1.0035, yfact=1.0)
    
    rings.append(c)

In [ ]:
mls = MultiLineString([r for r in rings])
layer = gp.make_like(mls, drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(layer)
# sk.vpype('linesort')
sk.display()

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/195_do_not_repeat.svg')

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
?gp.RegPolygon

In [ ]:
n_iters = 280
low = 0.9999
high = 1.0007
mu=0
sigma=0.00015

# c = Point((0,0)).buffer(20, resolution=120).boundary
sq = Point((0,0)).buffer(20, resolution=1).boundary
sq = gp.RegPolygon(Point(0,0), n_corners=6).poly.boundary
c =  Polygon([sq.interpolate(x, normalized=True) for x in np.linspace(0, 1, 80)]).boundary
rings = [c]
current_ring = np.array([x for x in c.coords])
rands = np.random.uniform(low, high, size=current_ring.shape)
# rands = np.random.lognormal(mean=mu, sigma=sigma, size=current_ring.shape)
for i in range(n_iters):
    current_ring = np.array([x for x in c.coords])
    current_ring *= rands
    current_ring += np.ones_like(current_ring) * np.array([0.05, -0.02])
    c = LineString(current_ring)
#     c = sa.skew(c, xs=0.1)
    c = sa.rotate(c, 0.6)
    c = sa.scale(c, xfact=1.0035, yfact=1.011)
    
    rings.append(c)

In [ ]:
mls = MultiLineString([r for r in rings])
layer = gp.make_like(mls, drawbox)

In [ ]:
l = layer[0]

In [ ]:
all_pts = [MultiPoint(l.coords) for l in layer]

In [ ]:
ps = []
for p in all_pts:
    ps += list(p)

In [ ]:
tris = so.triangulate(MultiPoint(ps))

In [ ]:
tries = gp.merge_Polygons(tris).boundary

In [ ]:
tries = gp.merge_LineStrings([t for t in tries if t.length < 10])

In [ ]:
tries

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(tries)
sk.vpype('splitall linemerge -t 0.3 linemerge -t 0.6 linemerge -t 0.9 linesort')
sk.display()